In [5]:
import pandas as pd
import numpy as np

data=pd.read_csv(r'Csvs/companyEPS.csv',encoding='utf-8') # ,encoding='utf-8' 如果無法讀要先用txt轉檔
# https://www.796t.com/post/MjJid3M=.html
print("info():",data.info())
print(data)

# print('=======----------')

#def csv_2_xlsx(srcPath):
   # try:
   #    data = pd.DataFrame(pd.read_csv(srcPath, encoding='utf8', low_memory=False))
   # except:
   #     data = pd.DataFrame(pd.read_csv(srcPath, encoding='gbk', low_memory=False))
   # data.to_excel(srcPath[:-3] + 'xlsx', sheet_name='Sheet1', index=False)

# 缺失值填充
# data = data1.fillna(0)
# data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63958 entries, 0 to 63957
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   市場         63958 non-null  object 
 1   季度         63958 non-null  int64  
 2   公司代號       63958 non-null  object 
 3   公司名稱       63897 non-null  object 
 4   產業別        63958 non-null  object 
 5   基本每股盈餘(元)  63958 non-null  float64
 6   普通股每股面額    63958 non-null  object 
 7   營業收入       63958 non-null  object 
 8   營業利益       63958 non-null  object 
 9   營業外收入及支出   63958 non-null  object 
 10  稅後淨利       63958 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 5.4+ MB
info(): None
         市場     季度  公司代號              公司名稱   產業別  基本每股盈餘(元)  \
0        上市  10201  1102        亞洲水泥股份有限公司  水泥工業       0.40   
1        上市  10201  1101        台灣水泥股份有限公司  水泥工業       0.38   
2        上市  10201  1104        環球水泥股份有限公司  水泥工業       0.30   
3        上市  10201  1108        幸福水泥股份有限公司  

In [ ]:
# 消除數字間的逗號 
data['稅後淨利']=data['稅後淨利'].str.replace(',','')
print(data['稅後淨利'])

# 將型態object轉成int 
data['稅後淨利']=pd.to_numeric(data['稅後淨利'], errors='coerce').fillna('0').astype('int64')
print(data['稅後淨利'])


In [ ]:
# 篩選需要的數據
filt = data['稅後淨利']>=0
print(data.loc[filt])

# d1=data.loc[:,['稅後淨利']]
# print("d1:",d1)
# d2=data.loc[filt]
# print("d2:",d2)

# data.loc可提取指定行數據 並印出想要的列
dataAll=data.loc[filt, ['公司代號', '公司名稱', '稅後淨利']]
print("提取指定行數據:",dataAll)

# 刪除重複欄位
dataAll = dataAll.drop_duplicates(subset='公司代號')
print("刪除重複欄位:",dataAll)

# 刪除缺失值
dataAll=dataAll.dropna()
print("刪除缺失值:",len(dataAll))

In [ ]:
dataAll.to_csv('dataPause/companyEPS_new.csv', index=False)

In [10]:
from selenium import webdriver  # 操作 browser 的 API
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait  # 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support import expected_conditions as EC  # 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.common.by import By  # 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.chrome.service import Service
from time import sleep  # 強制等待 (執行期間休息一下)

import pandas as pd
import numpy as np
import json  # 整理 json 使用的工具
import os, csv, re  # 執行 command 的時候用的

# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")              # 不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         # 最大化視窗
options.add_argument("--incognito")               # 開啟無痕模式
options.add_argument("--disable-popup-blocking ") # 禁用彈出攔截

# 使用 Chrome 的 WebDriver (含 options)
driver_path=Service('D:/chromedriver.exe')
driver=webdriver.Chrome(service=driver_path,options=options)

empty1=""
input1="1101"
url ='https://mops.twse.com.tw/mops/web/stapap1'

aveEpsth = ['公司名稱','職稱','姓名', '選任時持股', '目前持股', '設質股數', '設質股數佔持股比','目前持股合計','設質股數', '設質比例']
aveNum=len(aveEpsth)-1

def visit():
    # 前往指定連結
    global writeCount
    a=0
    writeCount=0
    driver.get(url)  
    f=open("Csvs/companyEPS_new.csv",encoding="utf8") 
    
    readCsv=pd.read_csv(f)
    dfCsv=pd.DataFrame(readCsv)
    
    df_ch=dfCsv['公司代號'].values.tolist()
    df_name=dfCsv['公司名稱'].values.tolist()
    
    print(len(df_ch))
        
    for df in df_ch:
        
        search(df)
        a+=1
        print("第筆:",a)
        print(df)
        downloadData(df,df_name)
        writeCount+=1
        #print(tdContent)
        
        #清空值
        driver.find_element(By.CSS_SELECTOR, "input[name='co_id'].textbox").clear()
            
        if a == len(df_ch):            
            print("-------------")
            print("爬蟲結束")     
    
def search(df): #search(df)
       
    txtInput = driver.find_element(By.CSS_SELECTOR, "input[name='co_id'].textbox")
    txtInput.send_keys(df)
    sleep(3)
    
    btnInput = driver.find_element(By.CSS_SELECTOR, "div.search input[type='button']")
    btnInput.click()
    sleep(3)
    

def downloadData(df,df_name):

    global th1,th2, tdContent1,tdContent2,thText,pp 
    
    thText=["公司名稱"]
    
    td1 = driver.find_elements(By.CSS_SELECTOR, "table.hasBorder tbody tr.odd td")  # html內基數行
    td2 = driver.find_elements(By.CSS_SELECTOR, "table.hasBorder tbody tr.even td")  # html內偶數行
           
    tdContent1 = [n.get_attribute('innerHTML').strip() for n in td1]
    tdContent2 = [n.get_attribute('innerHTML').strip() for n in td2]    

    th1 = tdContent1[:aveNum]
    th2 = tdContent2[:aveNum]    
    pp=df
    
    for j in thText:
        th1.insert(0, j)
        th2.insert(0, j)
        print("**************")
        print(th1)
        print(th2)
    print("*&&&&&&&&&&&&&&&&*")
    print(df_name[0])
        
    saveCsv(pp,df_name)
    
def saveCsv(pp,df_name):
    folderPath = 'Csvs'
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
            
    with open(f"{folderPath}/company_presidents_datas.csv", 'a', newline = '', encoding = 'utf-8') as csvfile:
        writer = csv.writer(csvfile)
        if writeCount == 0:
            writer.writerow(aveEpsth)
        
        print(len(tdContent1))
        print(len(tdContent2))  
        print("###############")
        print(pp)
        
        # 
        for i in range(0, len(tdContent1),aveNum):
                        
            c1 = tdContent1[i:i+aveNum]
            c2 = tdContent2[i:i+aveNum]
            
            # 因為不確定每個表格的基數偶數的行數量 所以用=0表示無此行
            if len(c1)==0 or len(c2)==0:
                print("沒有此行")
            else:   
                thText=str(pp)+' '+df_name[writeCount]
                c1.insert(0, thText)
                c2.insert(0, thText)
                writer.writerow(c1)
                writer.writerow(c2)
                
            print("((((((((()))))))))")   
            print(c1)
            print(c2)
        
    csvfile.close()
    
if __name__ == '__main__':
    visit()
    
driver.quit()


2304
第筆: 1
1102


WebDriverException: Message: chrome not reachable
  (Session info: chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x01047413+2389011]
	Ordinal0 [0x00FD9F61+1941345]
	Ordinal0 [0x00ECC520+836896]
	Ordinal0 [0x00EC0682+788098]
	Ordinal0 [0x00EC0EB8+790200]
	Ordinal0 [0x00EC2752+796498]
	Ordinal0 [0x00EBC0D9+770265]
	Ordinal0 [0x00ECD9D0+842192]
	Ordinal0 [0x00F23AE2+1194722]
	Ordinal0 [0x00F13F66+1130342]
	Ordinal0 [0x00EEE546+976198]
	Ordinal0 [0x00EEF456+980054]
	GetHandleVerifier [0x011F9632+1727522]
	GetHandleVerifier [0x012ABA4D+2457661]
	GetHandleVerifier [0x010DEB81+569713]
	GetHandleVerifier [0x010DDD76+566118]
	Ordinal0 [0x00FE0B2B+1968939]
	Ordinal0 [0x00FE5988+1989000]
	Ordinal0 [0x00FE5A75+1989237]
	Ordinal0 [0x00FEECB1+2026673]
	BaseThreadInitThunk [0x7643FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77447A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77447A4E+238]
